# export tiles for gain and loss
Based on [ee code](https://code.earthengine.google.com/8423c718d647f15ab3e9c31868489fd4?noload=true)

In [90]:
import ee
import geemap
%run utils.ipynb

In [2]:
# Trigger the authentication flow.
#ee.ServiceAccountCredentials(EE_ACCOUNT, EE_PRIVATE_KEY_FILE)
ee.Authenticate()


Successfully saved authorization token.


In [58]:
# Initialize the library.
ee.Initialize()

### Data Assets

In [70]:
gcs_bucket = 'mangrove_atlas'
dataset = ['mangrove-extent-gain', 'mangrove-extent-loss']

In [77]:
## Assets selection
# region for export
data_year_range = [2007, 2008, 2009, 2010, 2015, 2016, 2017, 2018, 2019, 2020]
region = ee.Geometry.Polygon([[[-180, 33],[-180, -34],[180, -34],[180, 33]]], None, False);
# gain
gain = ee.ImageCollection(
    f"projects/global-mangrove-watch/land-cover/{dataset[0]}")
# loss
loss = ee.ImageCollection(
    f"projects/global-mangrove-watch/land-cover/{dataset[1]}")

In [78]:
# styles
gain_ramp ='''
    <RasterSymbolizer>
        <ColorMap type="ramp" extended="false" >
        <ColorMapEntry color="#a6cb10" quantity="1" label="1"/>
        </ColorMap>
    </RasterSymbolizer>
'''
loss_ramp = '''
    <RasterSymbolizer>
        <ColorMap type="ramp" extended="false" >
        <ColorMapEntry color="#eb6240" quantity="1" label="1"/>
        </ColorMap>
    </RasterSymbolizer>
'''

### Vizualize the data prior exporting it

In [81]:
Map = geemap.Map(center=(-5, 39), zoom=12, basemap='HYBRID')
vis_params = {
    'min': 1,
    'max': 1,
    'palette': ['eb6240'],
}
Map.addLayer(ee.Image(gain.first()), vis_params, 'Gain', True, 0.5)
Map.addLayer(ee.Image(loss.first()), vis_params, 'Loss', True, 0.5)
Map

Map(center=[-5, 39], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

### Export map Tiles

In [82]:
# Create a list of tasks to export the map tiles
gain_export_tasks = exportMapTasks(gain, 'gain', gcs_bucket, data_year_range, 
            region, gain_ramp, 0, 12)

In [83]:
# Create a list of tasks to export the map tiles
loss_export_tasks = exportMapTasks(loss, 'loss', gcs_bucket, data_year_range, 
            region, loss_ramp, 0, 12)

In [84]:
tasks_list = gain_export_tasks + loss_export_tasks

In [85]:
len(tasks_list)

20

In [ ]:
batchExecute(tasks_list)

## Create vector tiles for gain and loss

In [86]:
my_asset = ee.ImageCollection('projects/global-mangrove-watch/land-cover/mangrove-extent-gain').first()

# Get nominal scale
ns = my_asset.projection().nominalScale().getInfo()
vector_reducer = {
            'reducer':ee.Reducer.countEvery(), 
                'geometry': region, 
                'scale': ns, 
                'geometryType': 'polygon', 
                'eightConnected': True, 
                'labelProperty': 'gain', 
                'bestEffort': True, 
                'maxPixels': 1e13, 
                'tileScale': 4, 
                'geometryInNativeProjection': False}

new_vector = ee.FeatureCollection(my_asset.reduceToVectors(**vector_reducer))

task = ee.batch.Export.table.toAsset(new_vector,'test_table', 'projects/global-mangrove-watch/test_vector', 1e6)
task.start()

# Export the vectorized FeatureCollection as a FeatureView asset.
task = ee.batch.Export.table.toFeatureView(**{
  'collection': new_vector,
  'assetId': 'projects/global-mangrove-watch/test_vector',
  'description': 'test_vector',
  'maxFeaturesPerTile': 1500,
  'thinningStrategy': 'HIGHER_DENSITY',
  'thinningRanking': ['REP_AREA DESC'],
  'zOrderRanking': ['REP_AREA DESC'],
})

task.start()

In [ ]:
Map = geemap.Map(center=(-7.998, 39.4767), zoom=9, basemap='HYBRID')
vis_params = {
    'min': 1,
    'max': 1,
    'palette': ['eb6240'],
}
Map.addLayer(my_asset, vis_params, 'gain', True, 1)
Map.addLayer(new_vector,{}, 'gain_vector', True, 1)
display(Map)